In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join as p_join

import seaborn as sns


##################################
## GLOBAL SETTINGS ###############
##################################
plt.rcParams["figure.figsize"] = (12,8)

In [2]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

from tqdm import tqdm


def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()   

[ Using Seed :  42  ]


In [3]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utils import utils

### Handle Function

In [4]:
#################################
#### MAKE TEST TRAIN SPLIT   ####
#################################
def create_dataloaders(CONFIG, X, Y):
    from sklearn.model_selection import train_test_split

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42)


    train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
    test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))

    train_dataloader = DataLoader(train_dataset,
                                  shuffle=True,
                                  num_workers=CONFIG['cpu_workers'],
                                  batch_size=CONFIG['train_batch_size'],
                                  drop_last=True)

    test_dataloader = DataLoader(test_dataset,
                                 shuffle=False,
                                 num_workers=CONFIG['cpu_workers'],
                                 batch_size=CONFIG['test_batch_size'],
                                 drop_last=False)

    return train_dataloader, test_dataloader

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 20.33it/s]


### Create models

**Conv + MLP**

In [5]:
class Block(nn.Module):
    def __init__(self, inp, out, ks):
        super().__init__()
        self.activ = F.gelu
        self.conv1 = nn.Conv1d(inp, out, ks, padding=1)
        self.bn1 = nn.BatchNorm1d(out)

    def forward(self, x):
        x = self.activ(self.bn1(self.conv1(x)))
        return x

class ConvMlpNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.drop_p = 0.1
        self.h_dim = 240
        self.h_dims = [1, 32, 64, self.h_dim]
        self.backbone = nn.Sequential(*[Block(self.h_dims[i], self.h_dims[i + 1], 3) for i in range(len(self.h_dims) - 1)])
        self.flat = nn.Flatten()
        self.drop = nn.Dropout(p=self.drop_p)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.lin = nn.Sequential(nn.Linear(self.h_dim, self.h_dim * 2),
                                 nn.BatchNorm1d(self.h_dim * 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim * 2, self.h_dim // 2),
                                 nn.BatchNorm1d(self.h_dim // 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim // 2, 4))

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = self.flat(x)
        x = self.drop(x)
        x = self.lin(x)
        return x

**Only MLP**

In [49]:
class MlpNet(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.drop_p = 0.1
        self.h_dim = 240
        self.drop = nn.Dropout(p=self.drop_p)
        self.lin = nn.Sequential(
                                 nn.Linear(in_features, in_features * 13),
                                 nn.BatchNorm1d(in_features * 13),
                                 nn.GELU(),
                                 nn.Linear(in_features * 13, self.h_dim),
                                 nn.BatchNorm1d(self.h_dim),
                                 nn.GELU(),
                                 nn.Linear(self.h_dim, self.h_dim * 2), 
                                 nn.BatchNorm1d(self.h_dim * 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim * 2, self.h_dim // 2),
                                 nn.BatchNorm1d(self.h_dim // 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim // 2, 4)
        )

    def forward(self, x):
        x = self.lin(x)
        return x

### Trainable params comparing

In [50]:
cnn_mlp = ConvMlpNet()
mlp = MlpNet(16)

utils.count_params(cnn_mlp), utils.count_params(mlp) 

[INFO]: Model "ConvMlpNet" has 228412 trainable parameters
[INFO]: Model "MlpNet" has 229676 trainable parameters


(None, None)

### Time comparation

In [51]:
from time import time

data_path = p_join('..', '..', 'data_2')
FOLDERS = [p_join(os.path.abspath(data_path), item) for item in os.listdir(data_path) if 'L=10.6' in item]


CONFIG = {'train_batch_size': 128,
          'test_batch_size': 128,
          'cpu_workers': 2}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)
X = np.expand_dims(X, axis=1)
train_dataloader_cnn, _ = create_dataloaders(CONFIG, X, Y)

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)
train_dataloader_mlp, _ = create_dataloaders(CONFIG, X, Y)


#########################
## SWITCH TO EVAL MODE ##
#########################
cnn_mlp = cnn_mlp.eval()
mlp = mlp.eval()

100%|███████████████████████████████████████████| 16/16 [00:02<00:00,  6.34it/s]


In [52]:
x, y = next(iter(train_dataloader_cnn))

In [53]:
%%timeit
with torch.no_grad():
    out = cnn_mlp(x)

36.8 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
x, y = next(iter(train_dataloader_mlp))

In [55]:
%%timeit
with torch.no_grad():
    out = mlp(x)

The slowest run took 4.42 times longer than the fastest. This could mean that an intermediate result is being cached.
14.1 ms ± 9.53 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [56]:
for x, y in tqdm(train_dataloader_cnn):
    with torch.no_grad():
        out = cnn_mlp(x)

100%|█████████████████████████████████████████| 399/399 [00:18<00:00, 21.38it/s]


In [57]:
for x, y in tqdm(train_dataloader_mlp):
    with torch.no_grad():
        out = mlp(x)

100%|█████████████████████████████████████████| 399/399 [00:08<00:00, 46.98it/s]


In [58]:
CONFIG = {'ephs': 35,
          'train_batch_size': 64,
          'test_batch_size': 16,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNet(16)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 1e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|█████████████████████████████████████████| 799/799 [01:35<00:00,  8.32it/s]


Epoch: 1/35, 	 total train loss: 1.013586215217958


100%|█████████████████████████████████████████| 800/800 [00:14<00:00, 54.43it/s]


Epoch: 1/35, 	 total score test: 56.825818551222945, [best score: 56.825818551222945]



100%|█████████████████████████████████████████| 799/799 [00:39<00:00, 20.04it/s]


...

Epoch: 34/35, 	 total score test: 79.6124091583965, [best score: 79.6124091583965]



100%|█████████████████████████████████████████| 799/799 [00:09<00:00, 88.77it/s]


Epoch: 35/35, 	 total train loss: 0.43275033965128684


100%|████████████████████████████████████████| 800/800 [00:02<00:00, 366.44it/s]

Epoch: 35/35, 	 total score test: 79.12010627490818, [best score: 79.6124091583965]



In [59]:
### Try more light MLP model
class MlpNet_s(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.drop_p = 0.1
        self.drop = nn.Dropout(p=self.drop_p)
        self.lin = nn.Sequential(
                                 nn.Linear(in_features, in_features * 2),
                                 nn.BatchNorm1d(in_features * 2),
                                 nn.GELU(),
                                 nn.Linear(in_features * 2, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Linear(in_features * 4, in_features * 8), 
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 8, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 4, 4)
        )

    def forward(self, x):
        x = self.lin(x)
        return x

CONFIG = {'ephs': 35,
          'train_batch_size': 64,
          'test_batch_size': 16,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNet(16)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 1e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|█████████████████████████████████████████| 799/799 [02:57<00:00,  4.49it/s]


Epoch: 1/35, 	 total train loss: 1.1041610408038163


100%|█████████████████████████████████████████| 800/800 [00:46<00:00, 17.08it/s]


Epoch: 1/35, 	 total score test: 53.1687114167383, [best score: 53.1687114167383]



100%|█████████████████████████████████████████| 799/799 [02:07<00:00,  6.27it/s]


...

Epoch: 34/35, 	 total score test: 71.87622098929437, [best score: 71.87622098929437]



100%|████████████████████████████████████████| 799/799 [00:04<00:00, 172.67it/s]


Epoch: 35/35, 	 total train loss: 0.6192399911274749


100%|████████████████████████████████████████| 800/800 [00:01<00:00, 408.18it/s]

Epoch: 35/35, 	 total score test: 72.17316558568415, [best score: 72.17316558568415]



In [60]:
utils.count_params(net)
pass

[INFO]: Model "MlpNet_s" has 20068 trainable parameters


In [61]:
### Try more light MLP model
class MlpNet_s(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.drop_p = 0.1
        self.drop = nn.Dropout(p=self.drop_p)
        self.lin = nn.Sequential(
                                 nn.Linear(in_features, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Linear(in_features * 4, in_features * 16),
                                 nn.BatchNorm1d(in_features * 16),
                                 nn.GELU(),
                                 nn.MaxPool1d(2),
                                 nn.Linear(in_features * 8, in_features * 8), 
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 8, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 4, 4)
        )

    def forward(self, x):
        x = self.lin(x)
        return x

CONFIG = {'ephs': 35,
          'train_batch_size': 64,
          'test_batch_size': 16,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNet_s(16)
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 1e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|███████████████████████████████████████████| 16/16 [00:02<00:00,  7.75it/s]


[INFO]: Model "MlpNet_s" has 43780 trainable parameters


100%|█████████████████████████████████████████| 799/799 [00:31<00:00, 25.77it/s]


Epoch: 1/35, 	 total train loss: 1.0604486602716363


100%|████████████████████████████████████████| 800/800 [00:05<00:00, 141.31it/s]


...

Epoch: 34/35, 	 total score test: 76.46323357036806, [best score: 76.46323357036806]



100%|████████████████████████████████████████| 799/799 [00:05<00:00, 137.04it/s]


Epoch: 35/35, 	 total train loss: 0.5203414125570816


100%|████████████████████████████████████████| 800/800 [00:02<00:00, 333.24it/s]

Epoch: 35/35, 	 total score test: 75.97093068687974, [best score: 76.46323357036806]



In [63]:
### Try more light MLP model
class MlpNet_s(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.drop_p = 0.1
        self.drop = nn.Dropout(p=self.drop_p)
        self.lin = nn.Sequential(
                                 nn.Linear(in_features, in_features * 8),
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                                 nn.Linear(in_features * 8, in_features * 32),
                                 nn.BatchNorm1d(in_features * 32),
                                 nn.GELU(),
                                 nn.MaxPool1d(2),
                                 nn.Linear(in_features * 16, in_features * 8), 
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 8, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 4, 4)
        )

    def forward(self, x):
        x = self.lin(x)
        return x

CONFIG = {'ephs': 35,
          'train_batch_size': 64,
          'test_batch_size': 16,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNet_s(16)
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 1e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|███████████████████████████████████████████| 16/16 [00:04<00:00,  3.55it/s]


[INFO]: Model "MlpNet_s" has 111300 trainable parameters


100%|█████████████████████████████████████████| 799/799 [01:15<00:00, 10.59it/s]


Epoch: 1/35, 	 total train loss: 1.0210404877221033


100%|█████████████████████████████████████████| 800/800 [00:32<00:00, 24.94it/s]


...

Epoch: 34/35, 	 total score test: 77.36188169102134, [best score: 77.79948425412206]



100%|█████████████████████████████████████████| 799/799 [00:28<00:00, 28.22it/s]


Epoch: 35/35, 	 total train loss: 0.4683025827694297


100%|█████████████████████████████████████████| 800/800 [00:17<00:00, 45.27it/s]

Epoch: 35/35, 	 total score test: 78.36211612096585, [best score: 78.36211612096585]



In [69]:
CONFIG = {'ephs': 50,
          'train_batch_size': 512,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNet_s(16)
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 2e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.37it/s]


[INFO]: Model "MlpNet_s" has 111300 trainable parameters


100%|███████████████████████████████████████████| 99/99 [00:03<00:00, 31.98it/s]


Epoch: 1/50, 	 total train loss: 1.0480722604375896


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 213.30it/s]


...

Epoch: 49/50, 	 total score test: 79.75306712510745, [best score: 79.75306712510745]



100%|███████████████████████████████████████████| 99/99 [00:02<00:00, 37.47it/s]


Epoch: 50/50, 	 total train loss: 0.3357537659731778


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 244.10it/s]

Epoch: 50/50, 	 total score test: 79.65148081581621, [best score: 79.75306712510745]



In [70]:
CONFIG = {'ephs': 100,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNet_s(16)
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 21.01it/s]


[INFO]: Model "MlpNet_s" has 111300 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:01<00:00, 26.46it/s]


Epoch: 1/100, 	 total train loss: 1.0682482573450829


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 309.09it/s]


...

Epoch: 99/100, 	 total score test: 81.11276080331328, [best score: 81.21434711260451]



100%|███████████████████████████████████████████| 49/49 [00:10<00:00,  4.74it/s]


Epoch: 100/100, 	 total train loss: 0.24293172481108685


100%|█████████████████████████████████████████| 200/200 [00:08<00:00, 23.70it/s]

Epoch: 100/100, 	 total score test: 81.11276080331328, [best score: 81.21434711260451]

